In [4]:
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

import argparse
import datetime
import os
import time
import random
import utils

import torch.nn as nn
import torchvision
import torch
import dataset.transforms as T

from engine import *

from dataset.coco_utils import get_coco, get_coco_kp
from dataset.group_by_aspect_ratio import GroupedBatchSampler, create_aspect_ratio_groups

import cv2

In [5]:
data_dir = '../datasets/coco/'
model_name = 'fasterrcnn_resnet50_fpn'
dataset_name = 'coco'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 8
epochs = 20
workers = 8
lr = 0.02
momentum = 0.9
weight_dacay = 1e-4
print_freq = 20
lr_step_size = 8
lr_steps = [8, 11]
lr_gamma = 0.1
resume = ''
test_only = True
result_dir = './results'
aspect_ratio_group_factor = 0
pretrained = True
distributed = False
parallel = False
world_size = 1
dist_url = 'env://'
mode = 'train'

In [6]:
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

In [7]:
def get_dataset(name, image_set, transforms=None):
    print('Loading Data...')
    paths = {'coco' : ('../datasets/coco/', get_coco, 91)}

    p, ds_fn, num_classes = paths[name]

    ds = ds_fn(p, image_set=image_set, transforms=transforms)
    print('Loading Data Finished')
    return ds, num_classes

In [8]:
#getting dataset - dataloader
print('creating dataloaders')

transforms = T.Compose([T.ToTensor(), T.RandomHorizontalFlip(0.5)])
dataset_train, num_classes = get_dataset('coco', 'train', transforms=transforms)
transforms = T.Compose([T.ToTensor()])
dataset_val = get_dataset('coco', 'val', transforms=transforms)
if distributed:
    train_sampler = torch.utils.data.distributed.DistributedSampler(dataset_train)
    val_sampler = torch.utils.data.distributed.DistributedSampler(dataset_val)
else:
    train_sampler = torch.utils.data.RandomSampler(dataset_train)
    val_sampler = torch.utils.data.SequentialSampler(dataset_val)

if aspect_ratio_group_factor >= 0:
    group_ids = create_aspect_ratio_groups(dataset_train, k=aspect_ratio_group_factor)
    train_batch_sampler = GroupedBatchSampler(train_sampler, group_ids, batch_size)
else:
    train_batch_sampler = torch.utils.data.BatchSampler(train_sampler, batch_size, drop_last=True)
data_loader_train = torch.utils.data.DataLoader(dataset_train, batch_sampler=train_batch_sampler, num_workers=workers, collate_fn=utils.collate_fn)
data_loader_val = torch.utils.data.DataLoader(dataset_val, batch_size = batch_size, sampler=val_sampler, num_workers=workers, collate_fn=utils.collate_fn)

creating dataloaders
Loading Data...
loading annotations into memory...
Done (t=14.18s)
creating index...
index created!
Loading Data Finished
Loading Data...
loading annotations into memory...
Done (t=1.68s)
creating index...
index created!
Loading Data Finished
Using [0, 1.0, inf] as bins for aspect ratio quantization
Count of instances per bin: [85308 31958]


In [ ]:
def _get_iou_types(model): #용도가뭐지???
    model_without_ddp = model
    if isinstance(model, torch.nn.parallel.DistributedDataParallel):
        model_without_ddp = model.module
    iou_types = ['bbox']
    if isinstance(model_without_ddp, torchvision.models.detection.MaskRCNN):
        iou_types.append('segm')
    if isinstance(model_without_ddp, torchvision.models.KeypointRCNN):
        iou_types.append('keypoints')
    return iou_types

In [ ]:
print('creating model')

model = torchvision.models.detection.__dict__[model_name](num_classes=num_classes, pretrained=pretrained)
device = torch.device(device)
model.to(device)

# # Distribute
#     model_without_ddp = model
#     if distributed:
#         model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[gpu])
#         model_without_ddp = model.module
#
#     # Parallel
#     if parallel:
#         print('Training parallel')
#         model = torch.nn.DataParallel(model).cuda()
#         model_without_ddp = model.module

params = [p for p in model.parameters() if p.requires_grad]
optim = torch.optim.SGD(params, lr=lr, momentum=momentum, weight_decay=weight_dacay)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optim, milestones=lr_steps, gamma=lr_gamma)

if resume:
    print('Resume training')
    ckpt = torch.load(resume, map_location='cpu')
    # model_without_ddp.load_state_dict(checkpoint['model'])
    optim.load_state_dict(ckpt['optim'])
    lr_scheduler.load_state_dict(ckpt['lr_scheduler'])

# if test_only:
#     evaluate(model, data_loader_val, device=device)

print('start training')

start_time = time.time()

st_epoch = 0

for epoch in range(st_epoch,epochs):
    train_one_epoch(model, optim, data_loader_train, device, epoch, print_freq)
    #######
    model.train()
    loss_arr = []

    warmup_lr_schedular = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader_train) - 1)

        warmup_lr_scheduler = utils.warmup_lr_scheduler(optim, warmup_iters, warmup_factor)

    for images, targets in data_loader_train:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

    loss_dict = model(images, targets)
    losses = sum(loss for loss in loss_dict.values())
    loss_arr += [losses]

    optim.zero_grad()
    losses.backward()
    optim.step()

    if warmup_lr_scheduler is not None:
        warmup_lr_scheduler.step()

    lr_scheduler.step()
    print('Losses Train {}'.format(loss_arr))
    #val

    with torch.no_grad():
        model.eval()
        loss_arr = []

        coco = get_coco_api_from_dataset(data_loader_val.dataset)
        iou_types = _get_iou_types(model)
        coco_evaluator = CocoEvaluator(coco, iou_types)

        for image, targets, in data_loader_val:
            image = list(img.to(device) for img in image)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            torch.cuda.synchronize()
            outputs = model(image)

            outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]

            res = {target['image_id'].item(): output for target, output in zip(targets, outputs)}

            coco_evaluator.update(res)

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_arr += [losses]
        print('Losses Val {}'.format(loss_arr))
        

    #########

    if result_dir and epoch % 10 == 0:
        utils.save_on_master({
            # 'model': model_without_ddp.state_dict(),
            'optimizer': optim.state_dict(),
            'lr_scheduler': lr_scheduler.state_dict()
        },
        os.path.join(result_dir, 'model_{}.pth'.format(epoch)))
    evaluate(model, data_loader_val, device=device)
total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))


creating model
start training
